### Transfer Learning 
1. 특정 조건에서 얻어진 어떤 지식을 다른 상황에 맞게 전이해서 활용하는 학습 방법 
    - 어떠한 데이터를 바탕으로 다양한 클래스를 구분하기 위해 학습되었음
    - 학습을 하기 위한 변수 중 어떤 이미지에도 적용할 수 있는 필터가 있을 것이고 이러한 필터는 상품 이미지 구분 같은 작업에도 충분히 사용가능 
    - 즉, 범용적인 형태를 구분할 수 있는 지식 학습한 필터가 되는 것 
2. 전이학습의 장점 
    - 데이터 부족을 어느 정도 해결 
    - 학습시간이 줄어듬 
        - 학습된 필터를 가지고 시작하기 때문에 현재 작업에 적합한 필터 조합만 추가적으로 학습이 필요하기 때문에 학습 시간이 적게 걸림 

### Data

In [1]:
!pip install wget

  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=47940dedb0bffafa3330b41591daa3b34acba6070b70f6101daa65c2c5ca6797
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [2]:
import os 
import wget

try:
    os.mkdir("images")
    os.mkdir("images/dogs")
    os.mkdir("images/cats")
except:
    pass

!wget https://i.kinja-img.com/gawker-media/image/upload/s--WFkXeene--/c_scale,f_auto,fl_progressive,q_80,w_800/ol9ceoqxidudap8owlwn.jpg -P images/dogs
!wget https://www.rspcansw.org.au/wp-content/uploads/2017/08/50_a-feature_dogs-and-puppies_mobile.jpg -P images/dogs  
!wget https://www.catster.com/wp-content/uploads/2018/05/A-gray-cat-crying-looking-upset.jpg -P images/cats
!wget https://www.scarymommy.com/wp-content/uploads/2018/01/c1.jpg?w=700 -P images/cats

--2021-01-29 08:45:41--  https://i.kinja-img.com/gawker-media/image/upload/s--WFkXeene--/c_scale,f_auto,fl_progressive,q_80,w_800/ol9ceoqxidudap8owlwn.jpg
Resolving i.kinja-img.com (i.kinja-img.com)... 151.101.66.166, 151.101.130.166, 151.101.2.166, ...
Connecting to i.kinja-img.com (i.kinja-img.com)|151.101.66.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 32099 (31K) [image/jpeg]
Saving to: ‘images/dogs/ol9ceoqxidudap8owlwn.jpg’

ol9ceoqxidudap8owlw 100%[===================>]  31.35K  --.-KB/s    in 0.002s  

2021-01-29 08:45:41 (12.3 MB/s) - ‘images/dogs/ol9ceoqxidudap8owlwn.jpg’ saved [32099/32099]

--2021-01-29 08:45:42--  https://www.rspcansw.org.au/wp-content/uploads/2017/08/50_a-feature_dogs-and-puppies_mobile.jpg
Resolving www.rspcansw.org.au (www.rspcansw.org.au)... 101.0.86.38
Connecting to www.rspcansw.org.au (www.rspcansw.org.au)|101.0.86.38|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 130940 (128K) [image/jpeg]
S

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

In [5]:
img_dir = "./images"
img_data = dset.ImageFolder(img_dir, transforms.Compose([
            transforms.RandomSizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            ]))

print(img_data.classes)
print(img_data.class_to_idx)
print(img_data.imgs)

['cats', 'dogs']
{'cats': 0, 'dogs': 1}
[('./images/cats/A-gray-cat-crying-looking-upset.jpg', 0), ('./images/dogs/50_a-feature_dogs-and-puppies_mobile.jpg', 1), ('./images/dogs/ol9ceoqxidudap8owlwn.jpg', 1)]


/usr/local/lib/python3.6/dist-packages/torchvision/transforms/transforms.py:841: UserWarning: The use of the transforms.RandomSizedCrop transform is deprecated, please use transforms.RandomResizedCrop instead.
  "please use transforms.RandomResizedCrop instead.")


### Set Parameters

In [7]:
batch_size = 2
learning_rate = 0.001
num_epoch = 10
num_category = 2

In [9]:
train_loader = DataLoader(img_data, batch_size = batch_size, shuffle = True, num_workers = 2, drop_last = True)
# 데이터를 받아서 batch_size만큼 학습하고 데이터를 섞어주며 batch_size가 안되는 데이터는 버림 

for img, label in train_loader:
    print(img.size())
    print(label)
    
# x와 label의 shape를 불러

torch.Size([2, 3, 224, 224])
tensor([0, 1])


In [10]:
import torchvision.models as models
resnet = models.resnet50(pretrained = True)
# 이미지넷으로 이미 학습한 ResNet 모델을 불러옴 

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


In [11]:
for name, module in resnet.named_children():
    print(name)
    
# 불러온 모델에서 named_children이라는 함수를 사용하면 ResNet 모델의 직속 자식 노드를 불러옴 
# 밑의 내용은 분류 과정의 Layer를 출력 
# layer4까지는 이미지 데이터에서 특성을 추출하고 avgpool을 통해 평균을 내서 fc layer를 통과시켜 분류를 완료하는 것 

conv1
bn1
relu
maxpool
layer1
layer2
layer3
layer4
avgpool
fc


In [12]:
class Resnet(nn.Module):
    def __init__(self):
        super(Resnet, self).__init__()
        self.layer0 = nn.Sequential(*list(resnet.children())[0:-1])
        # named_children을 호출했던 것과 비슷하게 직속 자식 노드들을 불러오되 이름은 빼고 모듈만 불러오기 
        # 모듈은 전부 불러오고 fc층은 사용하지 않을 예정이라서 -1을 사용 
        self.layer1 = nn.Sequential(
            nn.Linear(2048, 500),
            nn.BatchNorm1d(500),
            nn.ReLU(),
            nn.Linear(500, num_category),
            nn.ReLU()
        )
        # layer1층은 avgpool층을 통과하고 나온 텐서를 입력값으로 받아 구분하고자 하는 카테고리의 개수대로 분류 
        
    def forward(self, x):
        out = self.layer0(x)
        out = out.view(batch_size, -1)
        # 앞의 연산을 통과하고 나온 4차원 텐서를 nn.Lineargㅏㅁ수에 맞게 2차원으로 변환해주는 것 
        out = self.layer1(out)
        return out 

In [13]:
# 학습의 대상이 되는 변수 범위를 정하기 

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

model = Resnet().to(device)

for params in model.layer0.parameters():
    params.require_grad = False
    # layer0는 이미 학습된 모델이기 때문에 변수들이 돌면서 기울기를 계산하기 않게 해줌 
    
for params in model.layer1.parameters():
    params.requires_grad = True
    # layer1은 layer0과 정반대로 진행 

# Colab환경에서 gpu를 사용했기 때문에 cuda:0가 출력 cpu를 사용한다면 cpu가 출력 

cuda:0


In [14]:
for m in model.children():
    print(m)
    # layer0에는 엄청 복잡한 모델이 들어가있는 것을 볼 수 있음 

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)


In [15]:
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.layer1.parameters(), lr = learning_rate)

### Train

In [17]:
for i in range(num_epoch):
    for j, [image, label] in enumerate(train_loader):
        x = image.to(device)
        y_ = label.to(device)
        
        optimizer.zero_grad()
        output = model.forward(x)
        loss = loss_func(output, y_)
        loss.backward()
        optimizer.step()
        
    if i % 10 == 9:
        print(loss)

tensor(0.2221, device='cuda:0', grad_fn=<NllLossBackward>)


### Test

In [20]:
model.eval()

correct = 0
total = 0

with torch.no_grad():
    for image, label in train_loader:
        x = image.to(device)
        y_ = label.to(device)
        
        output = model.forward(x)
        _, output_index = torch.max(output,1)
        
        total += label.size(0)
        correct += (output_index == y_).sum().float()
        
print('Accuracy : {}'.format(100*correct/total))

Accuracy : 100.0
